In [17]:
import cv2
import numpy as np
import math
import albumentations as A
import os
from os import path
import glob
from pathlib import Path
import random

import matplotlib.pyplot as plt

### Rain Augmentation

In [128]:
def add_rain(img, pth):
    img_path = Path(pth)
    img_rain = img.copy()
    
    transform = A.RandomRain(
         slant_range=(-15, 15),
         drop_length=50,
         drop_width=1,
         drop_color=(180, 180, 180),
         blur_value=7,
         brightness_coefficient=0.7,
         p=1.0)

    res = transform(image=img_rain)["image"]
    cv2.imwrite(f'D:\\UAV_obj_det\\datasets\\rain\\images\\test\\{img_path.name}', res)
    return res

In [132]:
for path in glob.glob(r'D:\UAV_obj_det\datasets\images\test\*.jpg'):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    add_rain(image, path)

### Rotation Augmentation

In [18]:
image = cv2.imread('img.jpg', cv2.IMREAD_COLOR)

In [21]:
def read_yolo_bboxes(filepath):
    bboxes = []
    with open(filepath, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) == 5:  # Проверяем, что строка валидна
                cls, x_center, y_center, width, height = map(float, parts)
                bboxes.append([ int(cls), x_center, y_center, width, height])
    return bboxes

In [23]:
def save_yolo_bboxes(filepath, bboxes):
    with open(filepath, 'w') as f:
        for bbox in bboxes:
            line = f"{bbox[0]} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f} {bbox[4]:.6f}\n"
            f.write(line)

In [63]:
def rotate_image_and_bboxes(img, bboxes, angle_degrees):
    
    h, w = img.shape[:2]
    #angle = random.choice(angle_degrees) - если нужен не конкретный угол, а случайный из списка
    angle = angle_degrees
    
    # Поворот изображения
    rotation_matrix = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
    rotated_img = cv2.warpAffine(img, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR)
    
    rotated_bboxes = []
    for bbox in bboxes:
        cls, x_center, y_center, bw, bh = bbox
        
        # Масштабирование YOLO-координат
        x_min = int((x_center - bw / 2) * w)
        y_min = int((y_center - bh / 2) * h)
        x_max = int((x_center + bw / 2) * w)
        y_max = int((y_center + bh / 2) * h)
        
        # Аффинное преобразование
        corners = np.array([
            [x_min, y_min],
            [x_max, y_min],
            [x_max, y_max],
            [x_min, y_max]
        ])
        
        corners = np.hstack([corners, np.ones((4, 1))])
        rotated_corners = np.dot(rotation_matrix, corners.T).T
        
        # Новые границы 
        x_min_new = max(0, int(np.min(rotated_corners[:, 0])))
        y_min_new = max(0, int(np.min(rotated_corners[:, 1])))
        x_max_new = min(w, int(np.max(rotated_corners[:, 0])))
        y_max_new = min(h, int(np.max(rotated_corners[:, 1])))
        
        # 5. Проверка на наличие бокса
        new_width = x_max_new - x_min_new
        new_height = y_max_new - y_min_new
        if new_width > 5 and new_height > 5:
            # в YOLO-формат
            new_x_center = (x_min_new + new_width / 2) / w
            new_y_center = (y_min_new + new_height / 2) / h
            new_width = new_width / w
            new_height = new_height / h
            
            rotated_bboxes.append([cls, new_x_center, new_y_center, new_width, new_height])
    
    return rotated_img, rotated_bboxes, angle

In [45]:
def augment_and_save(img_path, label_path, angle_degrees):
    
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    bboxes = read_yolo_bboxes(label_path)
    
    rotated_img, rotated_bboxes, ang = rotate_image_and_bboxes(img, bboxes, angle_degrees)
    
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    new_img_path = os.path.join('D:\\UAV_obj_det\\datasets\\', f'rotate_{ang}\\images', f"{base_name}_rotated.jpg")
    new_label_path = os.path.join('D:\\UAV_obj_det\\datasets\\', f'rotate_{ang}\\labels', f"{base_name}_rotated.txt")
    
    cv2.imwrite(new_img_path, cv2.cvtColor(rotated_img, cv2.COLOR_RGB2BGR))
    save_yolo_bboxes(new_label_path, rotated_bboxes)
    
    print(f"Сохранено: {new_img_path}, {new_label_path}")

In [47]:
test_images = glob.glob(r'D:\UAV_obj_det\datasets\images\test\*.jpg')
test_labels = glob.glob(r'D:\UAV_obj_det\datasets\labels\test\*.txt')

In [61]:
for img_path, lbl_path in zip(test_images, test_labels):
    for angle in [-25, -20, -15, -10, -5, 0, 5, 10, 15, 20, 25]:
        augment_and_save(img_path, lbl_path, angle)

Сохранено: D:\UAV_obj_det\datasets\rotate_-25\images\0_100_30_0_03307_rotated.jpg, D:\UAV_obj_det\datasets\rotate_-25\labels\0_100_30_0_03307_rotated.txt
Сохранено: D:\UAV_obj_det\datasets\rotate_-20\images\0_100_30_0_03307_rotated.jpg, D:\UAV_obj_det\datasets\rotate_-20\labels\0_100_30_0_03307_rotated.txt
Сохранено: D:\UAV_obj_det\datasets\rotate_-15\images\0_100_30_0_03307_rotated.jpg, D:\UAV_obj_det\datasets\rotate_-15\labels\0_100_30_0_03307_rotated.txt
Сохранено: D:\UAV_obj_det\datasets\rotate_-10\images\0_100_30_0_03307_rotated.jpg, D:\UAV_obj_det\datasets\rotate_-10\labels\0_100_30_0_03307_rotated.txt
Сохранено: D:\UAV_obj_det\datasets\rotate_-5\images\0_100_30_0_03307_rotated.jpg, D:\UAV_obj_det\datasets\rotate_-5\labels\0_100_30_0_03307_rotated.txt
Сохранено: D:\UAV_obj_det\datasets\rotate_0\images\0_100_30_0_03307_rotated.jpg, D:\UAV_obj_det\datasets\rotate_0\labels\0_100_30_0_03307_rotated.txt
Сохранено: D:\UAV_obj_det\datasets\rotate_5\images\0_100_30_0_03307_rotated.jpg, D